In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Optimized dtypes
market_dtypes = {
    'ItinID': 'int64',
    'MktID': 'int64',
    'MktCoupons': 'int8',
    'Year': 'int16',
    'Quarter': 'int8',
    'OriginAirportID': 'int32',
    'OriginAirportSeqID': 'int32',
    'OriginCityMarketID': 'int32',
    'Origin': 'str',
    'OriginCountry': 'str',
    'OriginStateFips': 'int8',
    'OriginState': 'str',
    'OriginStateName': 'str',
    'OriginWac': 'int8',
    'DestAirportID': 'int32',
    'DestAirportSeqID': 'int32',
    'DestCityMarketID': 'int32',
    'Dest': 'str',
    'DestCountry': 'str',
    'DestStateFips': 'int8',
    'DestState': 'str',
    'DestStateName': 'str',
    'DestWac': 'int8',
    'AirportGroup': 'str',
    'WacGroup': 'str',
    'TkCarrierChange': 'float32',
    'TkCarrierGroup': 'str',
    'OpCarrierChange': 'float32',
    'OpCarrierGroup': 'str',
    'RPCarrier': 'str',
    'TkCarrier': 'str',
    'OpCarrier': 'str',
    'BulkFare': 'float32',
    'Passengers': 'float32',
    'MktFare': 'float32',
    'MktDistance': 'float32',
    'MktDistanceGroup': 'int8',
    'MktMilesFlown': 'float32',
    'NonStopMiles': 'float32',
    'ItinGeoType': 'int8',
    'MktGeoType': 'int8',
}

# Define filters
major_carriers = ['DL', 'AA', 'UA', 'WN', 'AS', 'NK', 'F9', 'B6']

print("Loading DB1B Market data with chunk filtering...")

# Read and filter in chunks
chunk_size = 100_000
filtered_chunks = []
total_rows_read = 0
total_rows_kept = 0

for chunk in pd.read_csv(
    'DB1BMarket_2025_2.csv',
    dtype=market_dtypes,
    usecols=list(market_dtypes.keys()),
    chunksize=chunk_size,
    low_memory=False
):
    # Filter chunk
    filtered_chunk = chunk[
        (chunk['Passengers'] > 0) &
        (chunk['MktFare'] > 0) &
        (chunk['BulkFare'] == 0) &
        (chunk['OpCarrier'].isin(major_carriers))
    ]
    
    # Track progress
    total_rows_read += len(chunk)
    total_rows_kept += len(filtered_chunk)
    
    if len(filtered_chunk) > 0:
        filtered_chunks.append(filtered_chunk)
    
    print(f"  Chunk: {len(chunk):,} rows → {len(filtered_chunk):,} kept "
          f"(Total: {total_rows_read:,} read, {total_rows_kept:,} kept)")

# Combine
print("\nCombining filtered chunks...")
market_df_big8 = pd.concat(filtered_chunks, ignore_index=True)

# Memory report
memory_gb = market_df_big8.memory_usage(deep=True).sum() / 1_073_741_824
filter_rate = (total_rows_kept / total_rows_read * 100) if total_rows_read > 0 else 0

print(f"\n✅ Loaded successfully!")
print(f"   Rows read: {total_rows_read:,}")
print(f"   Rows kept: {len(market_df_big8):,} ({filter_rate:.1f}%)")
print(f"   Columns: {len(market_df_big8.columns)}")
print(f"   Memory: {memory_gb:.2f} GB")
print(f"   Carriers: {', '.join(major_carriers)}")

# Preview
print("\n📊 Sample data:")
print(market_df_big8[['Origin', 'Dest', 'OpCarrier', 'Passengers', 'MktFare']].head())

Loading DB1B Market data with chunk filtering...
  Chunk: 100,000 rows → 99,608 kept (Total: 100,000 read, 99,608 kept)
  Chunk: 100,000 rows → 99,699 kept (Total: 200,000 read, 199,307 kept)
  Chunk: 100,000 rows → 87,932 kept (Total: 300,000 read, 287,239 kept)
  Chunk: 100,000 rows → 82,888 kept (Total: 400,000 read, 370,127 kept)
  Chunk: 100,000 rows → 83,856 kept (Total: 500,000 read, 453,983 kept)
  Chunk: 100,000 rows → 59,682 kept (Total: 600,000 read, 513,665 kept)
  Chunk: 100,000 rows → 6,365 kept (Total: 700,000 read, 520,030 kept)
  Chunk: 100,000 rows → 6,653 kept (Total: 800,000 read, 526,683 kept)
  Chunk: 100,000 rows → 7,662 kept (Total: 900,000 read, 534,345 kept)
  Chunk: 100,000 rows → 10,138 kept (Total: 1,000,000 read, 544,483 kept)
  Chunk: 100,000 rows → 29,851 kept (Total: 1,100,000 read, 574,334 kept)
  Chunk: 100,000 rows → 49,089 kept (Total: 1,200,000 read, 623,423 kept)
  Chunk: 100,000 rows → 9,325 kept (Total: 1,300,000 read, 632,748 kept)
  Chunk: 100

In [3]:
market_df_big8.columns

Index(['ItinID', 'MktID', 'MktCoupons', 'Year', 'Quarter', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCountry',
       'OriginStateFips', 'OriginState', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest',
       'DestCountry', 'DestStateFips', 'DestState', 'DestStateName', 'DestWac',
       'AirportGroup', 'WacGroup', 'TkCarrierChange', 'TkCarrierGroup',
       'OpCarrierChange', 'OpCarrierGroup', 'RPCarrier', 'TkCarrier',
       'OpCarrier', 'BulkFare', 'Passengers', 'MktFare', 'MktDistance',
       'MktDistanceGroup', 'MktMilesFlown', 'NonStopMiles', 'ItinGeoType',
       'MktGeoType'],
      dtype='object')

In [4]:
market_df_big8['OpCarrier'].unique().size

8

In [5]:
print("=== COLUMNS ===")
print(market_df_big8.columns.tolist())

print("\n=== DTYPES (default) ===")
print(market_df_big8.dtypes)

=== COLUMNS ===
['ItinID', 'MktID', 'MktCoupons', 'Year', 'Quarter', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCountry', 'OriginStateFips', 'OriginState', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCountry', 'DestStateFips', 'DestState', 'DestStateName', 'DestWac', 'AirportGroup', 'WacGroup', 'TkCarrierChange', 'TkCarrierGroup', 'OpCarrierChange', 'OpCarrierGroup', 'RPCarrier', 'TkCarrier', 'OpCarrier', 'BulkFare', 'Passengers', 'MktFare', 'MktDistance', 'MktDistanceGroup', 'MktMilesFlown', 'NonStopMiles', 'ItinGeoType', 'MktGeoType']

=== DTYPES (default) ===
ItinID                  int64
MktID                   int64
MktCoupons               int8
Year                    int16
Quarter                  int8
OriginAirportID         int32
OriginAirportSeqID      int32
OriginCityMarketID      int32
Origin                 object
OriginCountry          object
OriginStateFips          int8
Origin

In [6]:
market_df_big8['Quarter'].unique()

array([2], dtype=int8)

In [7]:
market_df_big8['Year'].unique()

array([2025], dtype=int16)

In [8]:
market_df_big8['OriginCountry'].unique()

array(['US'], dtype=object)

In [9]:
market_df_big8['DestCountry'].unique()

array(['US'], dtype=object)

In [10]:
# Dropping columns with single values that are obvious.
market_df_big8.drop(columns=['Quarter', 'Year', 'OriginCountry', 'DestCountry'], inplace=True)

In [31]:
# Check if columns have useful variance before dropping
for col in market_df_big8.columns:
    if col in market_df_big8.columns:
        unique_vals = market_df_big8[col].nunique()
        print(f"{col}: {unique_vals} unique values")
        if unique_vals <= 5:
            print(f"  → {market_df_big8[col].value_counts().to_dict()}")

ItinID: 3725041 unique values
MktID: 5736313 unique values
MktCoupons: 8 unique values
OriginAirportID: 261 unique values
OriginAirportSeqID: 261 unique values
OriginCityMarketID: 241 unique values
Origin: 261 unique values
OriginStateFips: 52 unique values
OriginState: 52 unique values
OriginStateName: 52 unique values
OriginWac: 52 unique values
DestAirportID: 260 unique values
DestAirportSeqID: 260 unique values
DestCityMarketID: 241 unique values
Dest: 260 unique values
DestStateFips: 52 unique values
DestState: 52 unique values
DestStateName: 52 unique values
DestWac: 52 unique values
AirportGroup: 152563 unique values
WacGroup: 41104 unique values
TkCarrierChange: 2 unique values
  → {0.0: 5736259, 1.0: 54}
TkCarrierGroup: 65 unique values
OpCarrierChange: 1 unique values
  → {0.0: 5736313}
OpCarrierGroup: 46 unique values
RPCarrier: 21 unique values
TkCarrier: 17 unique values
OpCarrier: 8 unique values
BulkFare: 1 unique values
  → {0.0: 5736313}
Passengers: 367 unique values
M

In [26]:
# Identify columns to drop
columns_to_drop = market_df_big8.columns[market_df_big8.columns.str.endswith('Fips')]
print(columns_to_drop)

Index(['OriginStateFips', 'DestStateFips'], dtype='object')


In [27]:
# Drop the identified columns
market_df_big8.drop(columns=columns_to_drop)

,ItinID,MktID,MktCoupons,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginState,OriginStateName,OriginWac,...,OpCarrier,BulkFare,Passengers,MktFare,MktDistance,MktDistanceGroup,MktMilesFlown,NonStopMiles,ItinGeoType,MktGeoType
0,202526327466,20252632746601,1,13930,1393008,30977,ORD,IL,Illinois,41,...,WN,0.0,1.0,283.619995,1005.0,3,1005.0,1005.0,2,2
1,202526327466,20252632746602,1,13204,1320402,31454,MCO,FL,Florida,33,...,WN,0.0,1.0,279.380005,990.0,2,990.0,990.0,2,2
2,202526327467,20252632746701,1,13930,1393008,30977,ORD,IL,Illinois,41,...,WN,0.0,1.0,298.230011,1005.0,3,1005.0,1005.0,2,2
3,202526327467,20252632746702,1,13204,1320402,31454,MCO,FL,Florida,33,...,WN,0.0,1.0,293.769989,990.0,2,990.0,990.0,2,2
4,202526327468,20252632746803,1,13303,1330303,32467,MIA,FL,Florida,33,...,WN,0.0,1.0,183.220001,1182.0,3,1182.0,1182.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5736308,202521059351,20252105935101,2,14107,1410702,30466,PHX,AZ,Arizona,81,...,AA,0.0,4.0,138.000000,1987.0,4,1987.0,1972.0,2,2
5736309,202521059352,20252105935201,2,14107,1410702,30466,PHX,AZ,Arizona,81,...,AA,0.0,1.0,143.000000,1987.0,4,1987.0,1972.0,2,2
5736310,202521059353,20252105935301,2,14107,1410702,30466,PHX,AZ,Arizona,81,...,AA,0.0,1.0,149.000000,1987.0,4,1987.0,1972.0,2,2
5736311,202521059354,20252105935401,2,14107,1410702,30466,PHX,AZ,Arizona,81,...,AA,0.0,1.0,153.000000,1987.0,4,1987.0,1972.0,2,2


In [28]:
columns_to_drop = market_df_big8.columns[market_df_big8.columns.str.endswith('Wac')]
print(columns_to_drop)

Index(['OriginWac', 'DestWac'], dtype='object')


In [29]:
market_df_big8.drop(columns=columns_to_drop)

,ItinID,MktID,MktCoupons,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginStateFips,OriginState,OriginStateName,...,OpCarrier,BulkFare,Passengers,MktFare,MktDistance,MktDistanceGroup,MktMilesFlown,NonStopMiles,ItinGeoType,MktGeoType
0,202526327466,20252632746601,1,13930,1393008,30977,ORD,17,IL,Illinois,...,WN,0.0,1.0,283.619995,1005.0,3,1005.0,1005.0,2,2
1,202526327466,20252632746602,1,13204,1320402,31454,MCO,12,FL,Florida,...,WN,0.0,1.0,279.380005,990.0,2,990.0,990.0,2,2
2,202526327467,20252632746701,1,13930,1393008,30977,ORD,17,IL,Illinois,...,WN,0.0,1.0,298.230011,1005.0,3,1005.0,1005.0,2,2
3,202526327467,20252632746702,1,13204,1320402,31454,MCO,12,FL,Florida,...,WN,0.0,1.0,293.769989,990.0,2,990.0,990.0,2,2
4,202526327468,20252632746803,1,13303,1330303,32467,MIA,12,FL,Florida,...,WN,0.0,1.0,183.220001,1182.0,3,1182.0,1182.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5736308,202521059351,20252105935101,2,14107,1410702,30466,PHX,4,AZ,Arizona,...,AA,0.0,4.0,138.000000,1987.0,4,1987.0,1972.0,2,2
5736309,202521059352,20252105935201,2,14107,1410702,30466,PHX,4,AZ,Arizona,...,AA,0.0,1.0,143.000000,1987.0,4,1987.0,1972.0,2,2
5736310,202521059353,20252105935301,2,14107,1410702,30466,PHX,4,AZ,Arizona,...,AA,0.0,1.0,149.000000,1987.0,4,1987.0,1972.0,2,2
5736311,202521059354,20252105935401,2,14107,1410702,30466,PHX,4,AZ,Arizona,...,AA,0.0,1.0,153.000000,1987.0,4,1987.0,1972.0,2,2


In [32]:
# dropping col BulkFare because there's a single value
# there are no BulkFares in this dataset
market_df_big8.drop(columns=['MktDistanceGroup', 'BulkFare', 'TkCarrierChange', 'OpCarrierChange', 'BulkFare'], inplace=True) 

In [35]:
print(market_df_big8.shape)
market_df_big8.head()

(5736313, 33)


,ItinID,MktID,MktCoupons,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginStateFips,OriginState,OriginStateName,...,RPCarrier,TkCarrier,OpCarrier,Passengers,MktFare,MktDistance,MktMilesFlown,NonStopMiles,ItinGeoType,MktGeoType
0,202526327466,20252632746601,1,13930,1393008,30977,ORD,17,IL,Illinois,...,WN,WN,WN,1.0,283.619995,1005.0,1005.0,1005.0,2,2
1,202526327466,20252632746602,1,13204,1320402,31454,MCO,12,FL,Florida,...,WN,WN,WN,1.0,279.380005,990.0,990.0,990.0,2,2
2,202526327467,20252632746701,1,13930,1393008,30977,ORD,17,IL,Illinois,...,WN,WN,WN,1.0,298.230011,1005.0,1005.0,1005.0,2,2
3,202526327467,20252632746702,1,13204,1320402,31454,MCO,12,FL,Florida,...,WN,WN,WN,1.0,293.769989,990.0,990.0,990.0,2,2
4,202526327468,20252632746803,1,13303,1330303,32467,MIA,12,FL,Florida,...,WN,WN,WN,1.0,183.220001,1182.0,1182.0,1182.0,2,2


In [34]:
market_df_big8.to_csv('db1b_market_filtered.csv', index=False)

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create bidirectional market
# market_df_big9['Market'] = market_df_big9.apply(lambda x: '-'.join(sorted([x['Origin'], x['Dest']])), axis=1)

# # Aggregate by market
# top_markets = market_df_big9.groupby('Market').agg({
#     'Passengers': 'sum',
#     'MktFare': 'mean',
#     'MktDistance': 'mean'
# }).reset_index()

# top_markets = top_markets.sort_values('Passengers', ascending=False).head(10)

# # Visualize
# fig, ax = plt.subplots(figsize=(12, 6))
# bars = ax.barh(top_markets['Market'], top_markets['Passengers'], color='steelblue')
# ax.set_xlabel('Total Passengers (Q2 2025)', fontsize=12)
# ax.set_title('Top 10 U.S. Domestic Markets by Passenger Volume', fontsize=14, fontweight='bold')
# ax.invert_yaxis()

# # Add value labels
# for bar in bars:
#     width = bar.get_width()
#     ax.text(width, bar.get_y() + bar.get_height()/2, f'{width:,.0f}', 
#             ha='left', va='center', fontsize=10)

# plt.tight_layout()
# plt.show()

# print(top_markets)